In [ ]:
%matplotlib inline

# Comparing different clustering algorithms on toy datasets


This example shows characteristics of different
clustering algorithms on datasets that are "interesting"
but still in 2D. With the exception of the last dataset,
the parameters of each of these dataset-algorithm pairs
has been tuned to produce good clustering results. Some
algorithms are more sensitive to parameter values than
others.

In [ ]:
import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

np.random.seed(0)

The sklearn.datasets package embeds some small toy datasets. There are three main kinds of dataset interfaces that can be used to get datasets depending on the desired type of dataset.

The dataset loaders. They can be used to load small standard datasets, described in the Toy datasets section.
The dataset fetchers. They can be used to download and load larger datasets, described in the Real world datasets section.

The dataset generation functions. They can be used to generate controlled synthetic datasets.
scikit-learn includes various random sample generators that can be used to build artificial datasets of controlled size and complexity.

In [ ]:
# ============
# Generate 6 datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500

# make_circles and make_moons generate 2d binary classification datasets that are challenging to certain algorithms
# they return X (array of shape[n_sample,2]) , y (label= 0 or 1 for class membership)
# 0< factor<1 scale factor between inner and outer circle 
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)
#print(noisy_circles[0][:15])

# noise: std deviation of gaussian noise added to the data
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
#print(noisy_moons)

# make_blobs create multiclass datasets by allocating each class one or more normally-distributed clusters of points.
# make_blobs provides greater control regarding the centers and standard deviations of each cluster,
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None
#print(no_structure)

# Anisotropicly distributed data: different properties in different directions
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
#print(X)

transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)
#print("after transformation",X_aniso)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state)


The last dataset is an example of a 'null' situation for clustering: the data is homogeneous, and there is no good
clustering. For this example, the null dataset uses the same parameters as the dataset in the row above it, which
represents a mismatch in the parameter values and the data structure

In [ ]:
# ========================================================
# plotting 9x6 plots for each dataset using all algorithms
# ========================================================
plt.figure(figsize=(9 * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,  hspace=.01)

plot_num = 1

#Set up default cluster parameters
default_base = {'quantile': .3,  'eps': .3, 'damping': .9,
                'preference': -200, 'n_neighbors': 10,  'n_clusters': 3}

# associate datasets with their parameters.
datasets = [
    (noisy_circles, {'damping': .77, 'preference': -240, 'quantile': .2, 'n_clusters': 2}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (varied, {'eps': .18, 'n_neighbors': 2}),
    (aniso, {'eps': .15, 'n_neighbors': 2}),
    (blobs, {}),
    (no_structure, {})]

#loop over all 6 datasets, extract algo params, fit and predict
for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])
. 
    # connectivity matrix for structured Ward. Computes the (weighted) graph of k-Neighbors for points in X
    # n_neighbors: Number of neighbors for each sample
    connectivity = kneighbors_graph(X, n_neighbors=params['n_neighbors'], include_self=False)
    
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create 9 different cluster objects for this dataset params
    # ============
    #1- Mean shift builds upon the concept of kernel density estimation (KDE)
    # KDE is a method to estimate the underlying distribution.The most popular is the Gaussian kernel
    # bandwith: the standard deviation of the distribution
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    
    #2- MiniBatchKMeans K-Means conducted on only a random sample of observations as opposed to all observations
    # n_clusters: the number of clusters to form
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    
    #3- AgglomerativeClustering uses the linkage parameter to determine the merging strategy to minimize
    # 1) variance of merged clusters (ward), 
    # 2) average of distance between observations from pairs of clusters (average), or
    # 3) maximum distance between observations from pairs of clusters (complete).
    # connectivity:
    ward = cluster.AgglomerativeClustering(n_clusters=params['n_clusters'], linkage='ward', connectivity=connectivity)
    
    #4- AgglomerativeClustering average linkage
    # affinity:  determines the distance metric used for linkage (minkowski, euclidean, etc.).
    average_linkage = cluster.AgglomerativeClustering(linkage="average", affinity="cityblock",
                                                      n_clusters=params['n_clusters'], connectivity=connectivity)
    
    #5- SpectralClustering to identify communities of nodes in a graph based on the edges connecting them
    # eigen_solver: The eigenvalue decomposition strategy to use. For a matrix A, if there exists a vector x which isn’t all 0’s and a scalar λ such that Ax = λx, 
    # then x is said to be an eigenvector of A with corresponding eigenvalue λ.
    spectral = cluster.SpectralClustering(n_clusters=params['n_clusters'], eigen_solver='arpack', affinity="nearest_neighbors")
    
    #6- DBScan (Density-based spatial clustering of applications with noise). 
    # groups together points that are close to each other based on a distance measurement and a minimum number of points.
    # eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other
    dbscan = cluster.DBSCAN(eps=params['eps'])
    
    #7- AffinityPropagation
    # damping:
    # preference:
    affinity_propagation = cluster.AffinityPropagation(damping=params['damping'], preference=params['preference'])
    
    #8- Birch
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    
    #9- Gaussian Mixture
    # covariance_type
    gmm = mixture.GaussianMixture(n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('MiniBatchKMeans', two_means),
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('SpectralClustering', spectral),
        ('Ward', ward),
        ('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )
    
    # loop over all 9 clustering algorithms
    for name, algorithm in clustering_algorithms:

        t0 = time.time()
        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding may not work as expected.",
                category=UserWarning)
            
            algorithm.fit(X) #fitting the algorithm
        t1 = time.time()
        

        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X) # predicting cluster class

        # assigning a color for each cluster class
        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a','#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        
        colors = np.append(colors, ["#000000"]) # add black color for outliers (if any)
        
        # create 6 rows 9 columns subplots
        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5) # sets x axis limits
        plt.ylim(-2.5, 2.5)
        plt.xticks(()) # Get locations and labels
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15, horizontalalignment='right')
        plot_num += 1

plt.show()